In [67]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format


In [32]:
#import excel from this path: "C:\Users\nicholas.t.norris\OneDrive - Vail Resorts Management Company\Documents\Python\vr-mkt-share-automation\240814 TAM Master V1 -ED.xlsx"
file_path = r"C:\Users\nicholas.t.norris\OneDrive - Vail Resorts Management Company\Documents\Python\vr-mkt-share-automation\240814 TAM Master V1 -ED.xlsx"
# Read the Excel file
mkt_raw = pd.read_excel(file_path, sheet_name='US 12 yr - by CBSA')

vri_raw = pd.read_excel(file_path, sheet_name='VRI data dump', index_col=0)

In [155]:
#only keep the 'Season', 'Ticket_2223_2324', 'Ticket type vVRI', 'Visitation Geo', 'ToRegion (VRI)', 'Visitation', columns from mkt
mkt = mkt_raw[['Season', 'Ticket_2223_2324', 'Ticket type vVRI', 'Visitation Geo', 'ToRegion (VRI)', 'Visitation']]
#in mkt, rename 'Ticket_2223_2324' to 'Ticket LOB', 'Ticket type vVRI' to 'Ticket Type', 'Visitation Geo' to 'VisGeo', 'ToRegion (VRI)' to 'Region', and 'Visitation' to 'SV'
mkt = mkt.rename(columns={
    'Ticket_2223_2324': 'Ticket LOB',
    'Ticket type vVRI': 'Ticket Type',
    'Visitation Geo': 'VisGeo',
    'ToRegion (VRI)': 'Region',
    'Visitation': 'SV'
})
#keep every column except 'Resort State' from vri_raw
vri = vri_raw.loc[:, vri_raw.columns != 'Resort State']
#in vri, rename 'Sub Region' to 'Region', 'Ticket Type Grouped' to 'Ticket Type', 'Line of Business (incl. EDP)' to 'LOB', 'Season Year' to 'Season', and 'SVs' to 'SV'
vri = vri.rename(columns={ 
    'Sub Region (adj.)': 'Region',
    'Ticket Type Grouped': 'Ticket Type',
    'Line Of Business  (incl. EDP)': 'LOB',
    'Season Year': 'Season',
    'SVs': 'SV',
    'Visitation Geo': 'VisGeo'
})
#replace all instances of (VRI) in the 'Ticket Type' column with '17MTN'
vri['VRI or Peak'] = vri['VRI or Peak'].str.replace('VRI', '17MTN', regex=False)


In [156]:
print(mkt)

        Season                        Ticket LOB Ticket Type         VisGeo  \
0      2023/24                       Season pass        Pass  International   
1      2023/24                       Paid ticket        Paid  International   
2      2023/24  Other (e.g. comp, special offer)        Paid  International   
3      2023/24                 Frequency product        Pass  International   
4      2023/24                       Paid ticket        Paid  International   
...        ...                               ...         ...            ...   
84653  2021/22                               NaN        Paid    Destination   
84654  2020/21                               NaN        Paid    Destination   
84655  2019/20                               NaN        Paid    Destination   
84656  2018/19                               NaN        Paid    Destination   
84657  2014/15                               NaN        Paid    Destination   

                  Region     SV  
0              No

In [157]:
print(vri)

                Region    Sub Region VRI or Peak            Resort  \
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
NaN       Mid Atlantic  Mid Atlantic          7S  Hidden Valley PA   
..                 ...           ...         ...               ...   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   
NaN  Pacific Northwest       WestxWB       17MTN      Stevens Pass   

    Ticket Type               LOB         VisGeo   Season         SV  
NaN        Paid              Paid    Destination  2022/23        NaN  
NaN        Paid  

### Do i need to remove comp, if so, how?

In [158]:
#use groupby to sum the 'Visitation' column in mkt by 'Season' where 'Ticket_2223_2324' is not 'Other (e.g. comp, special offer) and transpose the result
#mkt_total = mkt[mkt['Ticket_2223_2324'] != 'Other (e.g. comp, special offer)'].groupby('Season')['Visitation'].sum().reset_index()
mkt_total = mkt.groupby('Season')['SV'].sum().reset_index()
mkt_total = mkt_total.set_index('Season').T
#rename the index from 'SV' to 'Total industry visits'
mkt_total.index = ['Total industry visits']

In [159]:
print(mkt_total)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 56,904,226.77 55,969,413.46 52,983,180.22 52,157,828.63   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 54,208,793.34 52,765,389.63 58,625,979.45 50,540,814.84   

Season                      2020/21       2021/22       2022/23       2023/24  
Total industry visits 58,425,200.48 60,175,077.20 64,644,263.00 59,800,235.60  


In [160]:
#create a subset of mkt_total called mkt_pass_total where 'Ticket Type' is 'Pass'
mkt_pass_total = mkt[mkt['Ticket Type'] == 'Pass'].groupby('Season')['SV'].sum().reset_index()
mkt_pass_total = mkt_pass_total.set_index('Season').T
#rename the index from 'SV' to 'Total industry  visits'
mkt_pass_total.index = ['Total industry visits']
print(mkt_pass_total)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 18,356,524.97 20,099,343.95 18,257,615.93 18,747,171.58   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 20,330,000.24 19,790,779.54 23,239,819.02 19,924,122.02   

Season                      2020/21       2021/22       2022/23       2023/24  
Total industry visits 24,242,475.34 28,472,077.59 31,902,604.22 35,297,662.23  


In [161]:
#use gropuby to sum the 'SV' column in vri by 'Season' with a row for each value of 'VRI or Peak' and where 'Ticket Type' is 'Pass'
vri_pass = vri[vri['Ticket Type'] == 'Pass']
#print the sum of 'SV' in vri_pass
print(vri_pass['SV'].sum())
#group by 'Season' and 'Ticket Type' and sum the 'SV' column
vri_pass_total = vri_pass.groupby(['Season', 'VRI or Peak'])['SV'].sum().reset_index()
#pivot the result to have 'Season' as the columns
vri_pass_total = vri_pass_total.pivot(index='VRI or Peak', columns='Season', values='SV')
#add another row for the total of all seasons
#vri_pass_total.loc['Total'] = vri_pass_total.sum()

#reorder the rows to have '17MTN' first, 'Peak' second, and '7S' third
vri_pass_total = vri_pass_total.reindex(['17MTN', 'Peak', '7S'])
#rename the index values to 'VRI Visits (17MTN)', 'VRI Visits (Peak)', and 'VRI Visits (7S)'
vri_pass_total.index = ['VRI Visits (17MTN)', 'VRI Visits (Peak)', 'VRI Visits (7S)']
print(vri_pass_total)

49149832.07
Season                  2018/19      2019/20      2020/21      2021/22  \
VRI Visits (17MTN) 6,187,427.02 5,243,225.99 7,138,060.00 8,230,921.01   
VRI Visits (Peak)           NaN   792,886.00   885,240.00   975,067.00   
VRI Visits (7S)             NaN          NaN          NaN          NaN   

Season                  2022/23      2023/24  
VRI Visits (17MTN) 8,811,656.02 8,377,906.03  
VRI Visits (Peak)  1,084,401.00 1,002,793.00  
VRI Visits (7S)      229,853.00   190,396.00  


In [162]:
pass_total = pd.concat([mkt_total, vri_pass_total], axis=0, ignore_index=False)
print(pass_total)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 56,904,226.77 55,969,413.46 52,983,180.22 52,157,828.63   
VRI Visits (17MTN)              NaN           NaN           NaN           NaN   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 54,208,793.34 52,765,389.63 58,625,979.45 50,540,814.84   
VRI Visits (17MTN)              NaN           NaN  6,187,427.02  5,243,225.99   
VRI Visits (Peak)               NaN           NaN           NaN    792,886.00   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   

Season                      2020/21       2021/22       2022/23       2023/24  
Total industry visits 58,425,200.48 60,175,077.20 64,644,263.00 59,800,235.60  
VRI Visits (17MTN)     7,138

In [163]:
#I have index values of 'SV', '7S', 'Peak' and 'VRI'. For '7S', 'Peak', and 'VRI', I want to take the value and divide by the value in 'SV' for each season
pass_total.loc['VRI Share (17MTN)'] = pass_total.loc['VRI Visits (17MTN)'] / pass_total.loc['Total industry visits']
pass_total.loc['VRI Share (Peak)'] = pass_total.loc['VRI Visits (Peak)'] / pass_total.loc['Total industry visits']
pass_total.loc['VRI Share (7S)'] = pass_total.loc['VRI Visits (7S)'] / pass_total.loc['Total industry visits']
#now I want to rename the index values to '7S %', 'Peak %', and 'VRI %'


print(pass_total)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 56,904,226.77 55,969,413.46 52,983,180.22 52,157,828.63   
VRI Visits (17MTN)              NaN           NaN           NaN           NaN   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   
VRI Share (17MTN)               NaN           NaN           NaN           NaN   
VRI Share (Peak)                NaN           NaN           NaN           NaN   
VRI Share (7S)                  NaN           NaN           NaN           NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 54,208,793.34 52,765,389.63 58,625,979.45 50,540,814.84   
VRI Visits (17MTN)              NaN           NaN  6,187,427.02  5,243,225.99   
VRI Visits (Peak)               NaN           NaN           NaN    792,886.00   
VRI Visits (7S)            

In [164]:
def generate_table(
    mkt_df,
    vri_df,
    filters=None,
    group_column='VRI or Peak',
    total_label='Total industry visits',
    metric='SV',
    group_order=['17MTN', 'Peak', '7S'],
    rename_index_map=None
):
    """
    Generate a summary DataFrame with optional filters and share calculations.

    Parameters:
    - mkt_df, vri_df: DataFrames
    - filters: dict of {column_name: filter_value or None}
    - group_column: column in vri_df to group by (e.g. 'VRI or Peak')
    - total_label: row label for market total
    - metric: column to aggregate
    - group_order: list of group values to order rows
    - rename_index_map: optional dict to rename index labels

    Returns:
    - DataFrame with total visits and share calculations
    """

    def apply_filters(df, filters):
        if not filters:
            return df
        for col, val in filters.items():
            if val is not None:
                df = df[df[col] == val]
        return df

    # Apply filters
    mkt_subset = apply_filters(mkt_df, filters)
    vri_subset = apply_filters(vri_df, filters)

    # Market total
    mkt_total = mkt_subset.groupby('Season')[metric].sum().reset_index()
    mkt_total = mkt_total.set_index('Season').T
    mkt_total.index = [total_label]

    # VRI group totals
    vri_grouped = vri_subset.groupby(['Season', group_column])[metric].sum().reset_index()
    vri_pivot = vri_grouped.pivot(index=group_column, columns='Season', values=metric)

    # Reorder and rename
    vri_pivot = vri_pivot.reindex(group_order)
    if rename_index_map:
        vri_pivot.index = [rename_index_map.get(i, i) for i in vri_pivot.index]

    # Combine and compute shares
    combined = pd.concat([mkt_total, vri_pivot], axis=0)
    for idx in vri_pivot.index:
        share_label = idx.replace('Visits', 'Share')
        combined.loc[share_label] = combined.loc[idx] / combined.loc[total_label]

    return combined


In [165]:
rename_map = {
    '17MTN': 'VRI Visits (17MTN)',
    'Peak': 'VRI Visits (Peak)',
    '7S': 'VRI Visits (7S)'
}

pass_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': 'Pass',  # <- apply this filter
        'Region': None          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(pass_df)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 18,356,524.97 20,099,343.95 18,257,615.93 18,747,171.58   
VRI Visits (17MTN)              NaN           NaN           NaN           NaN   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   
VRI Share (17MTN)               NaN           NaN           NaN           NaN   
VRI Share (Peak)                NaN           NaN           NaN           NaN   
VRI Share (7S)                  NaN           NaN           NaN           NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 20,330,000.24 19,790,779.54 23,239,819.02 19,924,122.02   
VRI Visits (17MTN)              NaN           NaN  6,187,427.02  5,243,225.99   
VRI Visits (Peak)               NaN           NaN           NaN    792,886.00   
VRI Visits (7S)            

In [166]:
paid_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': 'Paid',  # <- apply this filter
        'Region': None          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(paid_df)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 38,547,701.80 35,870,069.51 34,725,564.29 33,410,657.05   
VRI Visits (17MTN)              NaN           NaN           NaN           NaN   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   
VRI Share (17MTN)               NaN           NaN           NaN           NaN   
VRI Share (Peak)                NaN           NaN           NaN           NaN   
VRI Share (7S)                  NaN           NaN           NaN           NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 33,878,793.10 32,974,610.09 35,386,160.43 30,616,692.81   
VRI Visits (17MTN)              NaN           NaN  4,105,419.91  2,862,582.39   
VRI Visits (Peak)               NaN           NaN           NaN  1,140,387.00   
VRI Visits (7S)            

In [167]:
total_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': None          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_df)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 56,904,226.77 55,969,413.46 52,983,180.22 52,157,828.63   
VRI Visits (17MTN)              NaN           NaN           NaN           NaN   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   
VRI Share (17MTN)               NaN           NaN           NaN           NaN   
VRI Share (Peak)                NaN           NaN           NaN           NaN   
VRI Share (7S)                  NaN           NaN           NaN           NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 54,208,793.34 52,765,389.63 58,625,979.45 50,540,814.84   
VRI Visits (17MTN)              NaN           NaN 10,292,846.93  8,105,808.38   
VRI Visits (Peak)               NaN           NaN           NaN  1,933,273.00   
VRI Visits (7S)            

In [168]:
total_rockies_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': 'Rocky Mountain'          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_rockies_df)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 19,800,403.78 20,691,898.42 20,277,027.89 21,774,811.05   
VRI Visits (17MTN)              NaN           NaN           NaN           NaN   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   
VRI Share (17MTN)               NaN           NaN           NaN           NaN   
VRI Share (Peak)                NaN           NaN           NaN           NaN   
VRI Share (7S)                  NaN           NaN           NaN           NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 21,301,363.02 20,368,388.73 23,911,656.26 19,740,651.96   
VRI Visits (17MTN)              NaN           NaN  6,909,744.96  5,451,481.15   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)            

In [169]:
total_midwest_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': 'Midwest'          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_midwest_df)

Season                     2012/13      2013/14      2014/15      2015/16  \
Total industry visits 7,273,465.25 7,679,353.24 6,962,697.83 5,498,044.62   
VRI Visits (17MTN)             NaN          NaN          NaN          NaN   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN   
VRI Share (17MTN)              NaN          NaN          NaN          NaN   
VRI Share (Peak)               NaN          NaN          NaN          NaN   
VRI Share (7S)                 NaN          NaN          NaN          NaN   

Season                     2016/17      2017/18      2018/19      2019/20  \
Total industry visits 5,403,272.18 6,229,875.24 6,470,371.07 6,369,462.05   
VRI Visits (17MTN)             NaN          NaN   324,791.02   319,623.00   
VRI Visits (Peak)              NaN          NaN          NaN   350,646.00   
VRI Visits (7S)                NaN          NaN          NaN          NaN  

In [170]:
total_midatlantic_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': 'Mid Atlantic'          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_midatlantic_df)

Season                     2012/13      2013/14      2014/15      2015/16  \
Total industry visits 5,155,137.88 5,769,158.07 5,673,328.03 3,956,576.55   
VRI Visits (17MTN)             NaN          NaN          NaN          NaN   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN   
VRI Share (17MTN)              NaN          NaN          NaN          NaN   
VRI Share (Peak)               NaN          NaN          NaN          NaN   
VRI Share (7S)                 NaN          NaN          NaN          NaN   

Season                     2016/17      2017/18      2018/19      2019/20  \
Total industry visits 4,184,432.41 4,160,753.52 4,262,269.10 3,835,312.00   
VRI Visits (17MTN)             NaN          NaN          NaN          NaN   
VRI Visits (Peak)              NaN          NaN          NaN   536,652.00   
VRI Visits (7S)                NaN          NaN          NaN          NaN  

In [171]:
total_northeast_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': 'Northeast'          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_northeast_df)

Season                      2012/13       2013/14       2014/15      2015/16  \
Total industry visits 13,333,571.53 13,385,941.14 13,332,252.17 9,346,316.20   
VRI Visits (17MTN)              NaN           NaN           NaN          NaN   
VRI Visits (Peak)               NaN           NaN           NaN          NaN   
VRI Visits (7S)                 NaN           NaN           NaN          NaN   
VRI Share (17MTN)               NaN           NaN           NaN          NaN   
VRI Share (Peak)                NaN           NaN           NaN          NaN   
VRI Share (7S)                  NaN           NaN           NaN          NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 11,936,450.77 11,986,511.05 12,445,257.71 11,433,968.99   
VRI Visits (17MTN)              NaN           NaN    965,449.94    865,115.45   
VRI Visits (Peak)               NaN           NaN           NaN  1,045,975.00   
VRI Visits (7S)                 NaN

In [172]:
total_pacific_southwest_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': 'Pacific Southwest'          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_pacific_southwest_df)

Season                     2012/13      2013/14      2014/15      2015/16  \
Total industry visits 7,140,140.78 5,055,859.79 4,737,645.60 7,376,469.04   
VRI Visits (17MTN)             NaN          NaN          NaN          NaN   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN   
VRI Share (17MTN)              NaN          NaN          NaN          NaN   
VRI Share (Peak)               NaN          NaN          NaN          NaN   
VRI Share (7S)                 NaN          NaN          NaN          NaN   

Season                     2016/17      2017/18      2018/19      2019/20  \
Total industry visits 7,285,058.93 6,169,866.33 7,482,368.81 5,880,240.36   
VRI Visits (17MTN)             NaN          NaN 1,755,698.00 1,221,224.63   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN  

In [173]:
total_pacific_northwest_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': 'Pacific Northwest'          # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_pacific_northwest_df)

Season                     2012/13      2013/14      2014/15      2015/16  \
Total industry visits 4,201,507.55 3,387,202.79 2,000,228.70 4,205,611.16   
VRI Visits (17MTN)             NaN          NaN          NaN          NaN   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN   
VRI Share (17MTN)              NaN          NaN          NaN          NaN   
VRI Share (Peak)               NaN          NaN          NaN          NaN   
VRI Share (7S)                 NaN          NaN          NaN          NaN   

Season                     2016/17      2017/18      2018/19      2019/20  \
Total industry visits 4,098,216.04 3,849,994.77 4,054,056.50 3,281,179.47   
VRI Visits (17MTN)             NaN          NaN   337,163.01   248,364.15   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN  

In [174]:
total_local_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': None,
        'VisGeo': 'Local'         # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_local_df)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 40,187,397.03 39,885,874.31 36,341,158.62 35,359,174.10   
VRI Visits (17MTN)              NaN           NaN           NaN           NaN   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   
VRI Share (17MTN)               NaN           NaN           NaN           NaN   
VRI Share (Peak)                NaN           NaN           NaN           NaN   
VRI Share (7S)                  NaN           NaN           NaN           NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 37,127,058.46 36,188,011.01 40,364,673.02 34,548,038.27   
VRI Visits (17MTN)              NaN           NaN  4,983,954.41  3,803,048.68   
VRI Visits (Peak)               NaN           NaN           NaN  1,933,273.00   
VRI Visits (7S)            

In [175]:
total_destination_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': None,
        'VisGeo': 'Destination'         # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_destination_df)

Season                      2012/13       2013/14       2014/15       2015/16  \
Total industry visits 14,918,135.49 14,529,676.67 14,944,880.19 15,209,551.06   
VRI Visits (17MTN)              NaN           NaN           NaN           NaN   
VRI Visits (Peak)               NaN           NaN           NaN           NaN   
VRI Visits (7S)                 NaN           NaN           NaN           NaN   
VRI Share (17MTN)               NaN           NaN           NaN           NaN   
VRI Share (Peak)                NaN           NaN           NaN           NaN   
VRI Share (7S)                  NaN           NaN           NaN           NaN   

Season                      2016/17       2017/18       2018/19       2019/20  \
Total industry visits 15,523,466.62 15,026,739.87 16,833,342.32 14,744,451.47   
VRI Visits (17MTN)              NaN           NaN  4,920,508.69  3,981,998.28   
VRI Visits (Peak)               NaN           NaN           NaN          0.00   
VRI Visits (7S)            

In [176]:
total_international_df = generate_table(
    mkt_df=mkt,
    vri_df=vri,
    filters={
        'Ticket Type': None,  # <- apply this filter
        'Region': None,
        'VisGeo': 'International'         # <- do not filter by region
    },
    group_column='VRI or Peak',
    total_label='Total industry visits',
    rename_index_map=rename_map
)
print(total_international_df)

Season                     2012/13      2013/14      2014/15      2015/16  \
Total industry visits 1,798,694.25 1,553,862.49 1,697,141.41 1,589,103.47   
VRI Visits (17MTN)             NaN          NaN          NaN          NaN   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN   
VRI Share (17MTN)              NaN          NaN          NaN          NaN   
VRI Share (Peak)               NaN          NaN          NaN          NaN   
VRI Share (7S)                 NaN          NaN          NaN          NaN   

Season                     2016/17      2017/18      2018/19      2019/20  \
Total industry visits 1,558,268.26 1,550,638.75 1,427,964.11 1,248,325.09   
VRI Visits (17MTN)             NaN          NaN   388,383.83   320,761.42   
VRI Visits (Peak)              NaN          NaN          NaN         0.00   
VRI Visits (7S)                NaN          NaN          NaN          NaN  

## create the Pass/Paid by Region

In [181]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = ['Pass', 'Paid', None]  # None means "Total"

reg_tt = {}

for region in regions:
    for ticket_type in ticket_types:
        label = f"{region} - {ticket_type or 'Total'}"
        print(f"Generating table for: {label}")
        
        try:
            table = generate_table(
                mkt_df=mkt,
                vri_df=vri,
                filters={
                    'Region': region,
                    'Ticket Type': ticket_type
                },
                group_column='VRI or Peak',
                total_label='Total industry visits',
                rename_index_map=rename_map
            )
            reg_tt[label] = table  # Store in a dict with a descriptive key
        except Exception as e:
            print(f"⚠️ Failed for {label}: {e}")


Generating table for: Rocky Mountain - Pass
Generating table for: Rocky Mountain - Paid
Generating table for: Rocky Mountain - Total
Generating table for: Midwest - Pass
Generating table for: Midwest - Paid
Generating table for: Midwest - Total
Generating table for: Mid Atlantic - Pass
Generating table for: Mid Atlantic - Paid
Generating table for: Mid Atlantic - Total
Generating table for: Northeast - Pass
Generating table for: Northeast - Paid
Generating table for: Northeast - Total
Generating table for: Pacific Southwest - Pass
Generating table for: Pacific Southwest - Paid
Generating table for: Pacific Southwest - Total
Generating table for: Pacific Northwest - Pass
Generating table for: Pacific Northwest - Paid
Generating table for: Pacific Northwest - Total


In [ ]:
#print(reg_tt['Pacific Northwest - Pass'])

Season                     2012/13      2013/14    2014/15      2015/16  \
Total industry visits 1,663,687.48 1,425,746.11 845,960.40 1,524,788.39   
VRI Visits (17MTN)             NaN          NaN        NaN          NaN   
VRI Visits (Peak)              NaN          NaN        NaN          NaN   
VRI Visits (7S)                NaN          NaN        NaN          NaN   
VRI Share (17MTN)              NaN          NaN        NaN          NaN   
VRI Share (Peak)               NaN          NaN        NaN          NaN   
VRI Share (7S)                 NaN          NaN        NaN          NaN   

Season                     2016/17      2017/18      2018/19      2019/20  \
Total industry visits 1,537,070.72 1,556,811.53 1,736,166.01 1,304,715.66   
VRI Visits (17MTN)             NaN          NaN   191,036.00   145,285.00   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN   
VRI Share (17M

## Region x VisGeo

In [183]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

vis_geo_types = ['Local', 'Destination', 'International']  # Add None if you want "total"

reg_vg = {}

for region in regions:
    for visgeo in vis_geo_types:
        label = f"{region} - {visgeo}"
        print(f"Generating table for: {label}")
        
        try:
            table = generate_table(
                mkt_df=mkt,
                vri_df=vri,
                filters={
                    'Region': region,
                    'Ticket Type': None,  # No filter here
                    'VisGeo': visgeo      # This varies
                },
                group_column='VRI or Peak',
                total_label='Total industry visits',
                rename_index_map=rename_map
            )
            reg_vg[label] = table
        except Exception as e:
            print(f"⚠️ Failed for {label}: {e}")


Generating table for: Rocky Mountain - Local
Generating table for: Rocky Mountain - Destination
Generating table for: Rocky Mountain - International
Generating table for: Midwest - Local
Generating table for: Midwest - Destination
Generating table for: Midwest - International
Generating table for: Mid Atlantic - Local
Generating table for: Mid Atlantic - Destination
Generating table for: Mid Atlantic - International
Generating table for: Northeast - Local
Generating table for: Northeast - Destination
Generating table for: Northeast - International
Generating table for: Pacific Southwest - Local
Generating table for: Pacific Southwest - Destination
Generating table for: Pacific Southwest - International
Generating table for: Pacific Northwest - Local
Generating table for: Pacific Northwest - Destination
Generating table for: Pacific Northwest - International


In [ ]:
#print(reg_vg['Rocky Mountain - International'])

Season                     2012/13      2013/14      2014/15      2015/16  \
Total industry visits 1,153,341.08 1,057,013.40 1,189,449.56 1,159,547.56   
VRI Visits (17MTN)             NaN          NaN          NaN          NaN   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN   
VRI Share (17MTN)              NaN          NaN          NaN          NaN   
VRI Share (Peak)               NaN          NaN          NaN          NaN   
VRI Share (7S)                 NaN          NaN          NaN          NaN   

Season                     2016/17      2017/18    2018/19    2019/20  \
Total industry visits 1,094,893.57 1,083,867.30 896,784.18 768,874.48   
VRI Visits (17MTN)             NaN          NaN 332,689.81 274,343.68   
VRI Visits (Peak)              NaN          NaN        NaN        NaN   
VRI Visits (7S)                NaN          NaN        NaN        NaN   
VRI Share (17MTN) 

## Ticket Type x VisGeo

In [187]:
ticket_types = ['Pass', 'Paid']
vis_geo_types = ['Local', 'Destination', 'International']

tt_vg = {}

for ticket_type in ticket_types:
    for visgeo in vis_geo_types:
        label = f"{ticket_type} - {visgeo}"
        print(f"Generating table for: {label}")

        try:
            table = generate_table(
                mkt_df=mkt,
                vri_df=vri,
                filters={
                    'Region': None,           # No region filter
                    'Ticket Type': ticket_type,
                    'VisGeo': visgeo
                },
                group_column='VRI or Peak',
                total_label='Total industry visits',
                rename_index_map=rename_map
            )
            tt_vg[label] = table
        except Exception as e:
            print(f"⚠️ Failed for {label}: {e}")


Generating table for: Pass - Local
Generating table for: Pass - Destination
Generating table for: Pass - International
Generating table for: Paid - Local
Generating table for: Paid - Destination
Generating table for: Paid - International


In [ ]:
#print(tt_vg['Pass - International'])


Season                   2012/13    2013/14    2014/15    2015/16    2016/17  \
Total industry visits 366,832.95 434,477.97 464,307.27 488,269.70 538,186.75   
VRI Visits (17MTN)           NaN        NaN        NaN        NaN        NaN   
VRI Visits (Peak)            NaN        NaN        NaN        NaN        NaN   
VRI Visits (7S)              NaN        NaN        NaN        NaN        NaN   
VRI Share (17MTN)            NaN        NaN        NaN        NaN        NaN   
VRI Share (Peak)             NaN        NaN        NaN        NaN        NaN   
VRI Share (7S)               NaN        NaN        NaN        NaN        NaN   

Season                   2017/18    2018/19    2019/20    2020/21    2021/22  \
Total industry visits 461,811.15 475,633.07 444,534.65 229,285.99 367,164.14   
VRI Visits (17MTN)           NaN 223,333.10 207,270.57 102,202.88 203,085.35   
VRI Visits (Peak)            NaN        NaN       0.00       0.00       0.00   
VRI Visits (7S)              NaN       

## Region x TicketType x VisGeo

In [189]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = ['Local', 'Destination', 'International']

reg_tt_vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg_tt_vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: Rocky Mountain - Pass - Local
Generating table for: Rocky Mountain - Pass - Destination
Generating table for: Rocky Mountain - Pass - International
Generating table for: Rocky Mountain - Paid - Local
Generating table for: Rocky Mountain - Paid - Destination
Generating table for: Rocky Mountain - Paid - International
Generating table for: Midwest - Pass - Local
Generating table for: Midwest - Pass - Destination
Generating table for: Midwest - Pass - International
Generating table for: Midwest - Paid - Local
Generating table for: Midwest - Paid - Destination
Generating table for: Midwest - Paid - International
Generating table for: Mid Atlantic - Pass - Local
Generating table for: Mid Atlantic - Pass - Destination
Generating table for: Mid Atlantic - Pass - International
Generating table for: Mid Atlantic - Paid - Local
Generating table for: Mid Atlantic - Paid - Destination
Generating table for: Mid Atlantic - Paid - International
Generating table for: Northeast - 

In [ ]:
#print(reg_tt_vg['Rocky Mountain - Pass - Destination'])

Season                     2012/13      2013/14      2014/15      2015/16  \
Total industry visits 1,666,680.15 2,306,745.08 2,277,221.07 2,669,346.76   
VRI Visits (17MTN)             NaN          NaN          NaN          NaN   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN   
VRI Share (17MTN)              NaN          NaN          NaN          NaN   
VRI Share (Peak)               NaN          NaN          NaN          NaN   
VRI Share (7S)                 NaN          NaN          NaN          NaN   

Season                     2016/17      2017/18      2018/19      2019/20  \
Total industry visits 3,224,206.33 2,976,194.71 3,729,059.19 3,766,086.05   
VRI Visits (17MTN)             NaN          NaN 1,949,539.50 1,928,391.25   
VRI Visits (Peak)              NaN          NaN          NaN          NaN   
VRI Visits (7S)                NaN          NaN          NaN          NaN  

# Everything below this uses the same block of code, but just adjusts the filters. Use these

## Ticket Type

In [193]:
regions = [
    None
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = [None]

tt = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                tt[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: None - Pass - None
Generating table for: None - Paid - None


## Region

In [197]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = [None]
vis_geo_types = [None]

reg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: Rocky Mountain - None - None
Generating table for: Midwest - None - None
Generating table for: Mid Atlantic - None - None
Generating table for: Northeast - None - None
Generating table for: Pacific Southwest - None - None
Generating table for: Pacific Northwest - None - None


## VisGeo

In [200]:
regions = [
    None
]

ticket_types = [None]
vis_geo_types = ['Local', 'Destination', 'International']

vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: None - None - Local
Generating table for: None - None - Destination
Generating table for: None - None - International


## Region x Ticket Type

In [202]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = [None]

reg_tt = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg_tt[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: Rocky Mountain - Pass - None
Generating table for: Rocky Mountain - Paid - None
Generating table for: Midwest - Pass - None
Generating table for: Midwest - Paid - None
Generating table for: Mid Atlantic - Pass - None
Generating table for: Mid Atlantic - Paid - None
Generating table for: Northeast - Pass - None
Generating table for: Northeast - Paid - None
Generating table for: Pacific Southwest - Pass - None
Generating table for: Pacific Southwest - Paid - None
Generating table for: Pacific Northwest - Pass - None
Generating table for: Pacific Northwest - Paid - None


## Region x VisGeo

In [205]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = [None]
vis_geo_types = ['Local', 'Destination', 'International']

reg_vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg_vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: Rocky Mountain - None - Local
Generating table for: Rocky Mountain - None - Destination
Generating table for: Rocky Mountain - None - International
Generating table for: Midwest - None - Local
Generating table for: Midwest - None - Destination
Generating table for: Midwest - None - International
Generating table for: Mid Atlantic - None - Local
Generating table for: Mid Atlantic - None - Destination
Generating table for: Mid Atlantic - None - International
Generating table for: Northeast - None - Local
Generating table for: Northeast - None - Destination
Generating table for: Northeast - None - International
Generating table for: Pacific Southwest - None - Local
Generating table for: Pacific Southwest - None - Destination
Generating table for: Pacific Southwest - None - International
Generating table for: Pacific Northwest - None - Local
Generating table for: Pacific Northwest - None - Destination
Generating table for: Pacific Northwest - None - International


## Ticket Type x VisGeo

In [207]:
regions = [
    None
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = ['Local', 'Destination', 'International']

tt_vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                tt_vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: None - Pass - Local
Generating table for: None - Pass - Destination
Generating table for: None - Pass - International
Generating table for: None - Paid - Local
Generating table for: None - Paid - Destination
Generating table for: None - Paid - International


## Region x Ticket Type x VisGeo

In [209]:
regions = [
    'Rocky Mountain', 'Midwest', 'Mid Atlantic',
    'Northeast', 'Pacific Southwest', 'Pacific Northwest'
]

ticket_types = ['Pass', 'Paid']
vis_geo_types = ['Local', 'Destination', 'International']

reg_tt_vg = {}

for region in regions:
    for ticket_type in ticket_types:
        for visgeo in vis_geo_types:
            label = f"{region} - {ticket_type} - {visgeo}"
            print(f"Generating table for: {label}")

            try:
                table = generate_table(
                    mkt_df=mkt,
                    vri_df=vri,
                    filters={
                        'Region': region,
                        'Ticket Type': ticket_type,
                        'VisGeo': visgeo
                    },
                    group_column='VRI or Peak',
                    total_label='Total industry visits',
                    rename_index_map=rename_map
                )
                reg_tt_vg[label] = table
            except Exception as e:
                print(f"⚠️ Failed for {label}: {e}")


Generating table for: Rocky Mountain - Pass - Local
Generating table for: Rocky Mountain - Pass - Destination
Generating table for: Rocky Mountain - Pass - International
Generating table for: Rocky Mountain - Paid - Local
Generating table for: Rocky Mountain - Paid - Destination
Generating table for: Rocky Mountain - Paid - International
Generating table for: Midwest - Pass - Local
Generating table for: Midwest - Pass - Destination
Generating table for: Midwest - Pass - International
Generating table for: Midwest - Paid - Local
Generating table for: Midwest - Paid - Destination
Generating table for: Midwest - Paid - International
Generating table for: Mid Atlantic - Pass - Local
Generating table for: Mid Atlantic - Pass - Destination
Generating table for: Mid Atlantic - Pass - International
Generating table for: Mid Atlantic - Paid - Local
Generating table for: Mid Atlantic - Paid - Destination
Generating table for: Mid Atlantic - Paid - International
Generating table for: Northeast - 